In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
import numpy as np
import nltk

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [8]:
from defs import get_tokens
from defs import get_lemmas
nltk.download('punkt')
nltk.download('wordnet')
sms = pd.read_csv('datasets/sms_spam_no_header.csv', sep=',', names = ["type", "text"])
text_train, text_test, type_train, type_test = train_test_split(sms['text'], sms['type'], test_size = 0.3)
#bow stands for "Bag of Words"

bow = CountVectorizer(analyzer=get_lemmas).fit(text_train)
sms_bow = bow.transform(text_train)
tfidf = TfidfTransformer().fit(sms_bow)
sms_tfidf = tfidf.transform(sms_bow)
spam_detector = MultinomialNB().fit(sms_tfidf, type_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
msg = sms['text'][25]
msg_bow = bow.transform([msg])
msg_tfidf = tfidf.transform(msg_bow)

print('predicted: ', spam_detector.predict(msg_tfidf)[0])
print('expected: ', sms.type[25])

predicted:  ham
expected:  ham


In [11]:
msg = sms['text'][11]
msg_bow = bow.transform([msg])
msg_tfidf = tfidf.transform(msg_bow)

print('predicted: ', spam_detector.predict(msg_tfidf)[0])
print('expected: ', sms.type[11])

predicted:  spam
expected:  spam


In [12]:
predictions = spam_detector.predict(sms_tfidf)
print('accuracy', accuracy_score(sms['type'][:len(predictions)], predictions))

accuracy 0.7918482440399898


In [13]:
print(classification_report(sms['type'][:len(predictions)], predictions))

              precision    recall  f1-score   support

         ham       0.87      0.90      0.88      3382
        spam       0.13      0.10      0.11       519

    accuracy                           0.79      3901
   macro avg       0.50      0.50      0.50      3901
weighted avg       0.77      0.79      0.78      3901

